In [1]:
import os
import json
import pandas as pd
import sqlite3 as sl
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains as AC
import time

In [2]:
cycle_csvs = [
    {
        "csv": "bike_activities.csv",
        "label": 0,
        "mtb": 0
    },
    {
        "csv": "mtb_activities.csv",
        "label": 0,
        "mtb": 1
    },
    {
        "csv": "ebike_activities.csv",
        "label": 1,
        "mtb": 0
    },
    {
        "csv": "emtb_activities.csv",
        "label": 1,
        "mtb": 1
    }
]

csv = "ebike_activities.csv"
label = 1
mtb = 0

In [3]:
#building the url to catch the datastream, more or less data can be scraped
def buildQuery(activity):
    return "https://www.strava.com/activities/" + activity + "/streams" \
            "?stream_types%5B%5D=velocity_smooth" \
            "&stream_types%5B%5D=watts" \
            "&stream_types%5B%5D=heartrate" \
            "&stream_types%5B%5D=cadence" \
            "&stream_types%5B%5D=moving" \
            "&stream_types%5B%5D=distance" \
            "&stream_types%5B%5D=time" \
            "&stream_types%5B%5D=altitude" \
            "&stream_types%5B%5D=grade_smooth" \
            "&stream_types%5B%5D=grade_adjusted_distance"

In [4]:
#saving as csv
def saveData(df, activity):
    os.makedirs('data/', exist_ok=True)
    df.to_csv('data/' + activity + '.csv')
    
def db_get_ids():
    con = sl.connect("cycling.db")
    query = "SELECT DISTINCT activity_id FROM data_raw"
    return pd.read_sql(query, con).activity_id

In [5]:
driver = webdriver.Firefox()
action = AC(driver)
driver.get("https://www.strava.com/login")
username = driver.find_element(By.ID, "email")
password = driver.find_element(By.ID, "password")
username.send_keys("ds@uni.kn")
password.send_keys("dkwo20dk")
driver.find_element(By.ID, "login-button").click()

In [6]:
activity_ids = pd.read_csv(csv).keys().to_list()
activity_ids_existing = db_get_ids().to_list()
con = sl.connect("cycling.db")

In [7]:
for activity in activity_ids:
    if activity in activity_ids_existing:
        print("This id already exists in the database")
        continue
    query = buildQuery(activity)
    driver.get(query)
    #explicitly_wait would be more precise
    driver.implicitly_wait(5)
    try:
        driver.find_element(By.ID, "rawdata-tab").click()
    except Exception as e:
        print(e)
    driver.implicitly_wait(5)
    time.sleep(4)
    raw_data = driver.find_element(By.XPATH, "//pre[@class='data']").text
    data = pd.read_json(raw_data)
    #adding some metadata
    data = data.assign(activity_id = lambda x: activity)
    data = data.assign(label = lambda x: label)
    data = data.assign(mtb = lambda x: mtb)
    data = data.set_index(["time"])
    #saving to csv and sql
    saveData(data, activity)
    try:
        data.to_sql("data_raw", con, if_exists="append")
    except Exception as e:
        print(e)
print("finished!")

This id already exists in the database
This id already exists in the database
This id already exists in the database
This id already exists in the database
This id already exists in the database
This id already exists in the database
This id already exists in the database
This id already exists in the database
This id already exists in the database
This id already exists in the database
This id already exists in the database
This id already exists in the database
This id already exists in the database


/tmp/ipykernel_239753/1043656247.py:16: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_json(raw_data)


UNIQUE constraint failed: data_raw.activity_id, data_raw.time


/tmp/ipykernel_239753/1043656247.py:16: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_json(raw_data)


UNIQUE constraint failed: data_raw.activity_id, data_raw.time
UNIQUE constraint failed: data_raw.activity_id, data_raw.time
This id already exists in the database
This id already exists in the database
This id already exists in the database
This id already exists in the database
This id already exists in the database
This id already exists in the database
This id already exists in the database
finished!


/tmp/ipykernel_239753/1043656247.py:16: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  data = pd.read_json(raw_data)
